# Effective Groundwater Model Calibration

## Imports

In [1]:
import flopy
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Conceptual Model

![title](img/conceptual_model_1.png)

![title](img/conceptual_model_2.png)

![title](img/conceptual_model_3.png)

## Steady-State model setup 

We are creating a square model with:

* 2 layers
* 18 rows
* 18 columns
* GHB-Boundary (right side) 
* RIV-Boundary (left side)

### Set workspace for steady-state model

* and make shure that directory exists  
* save the starting path
* change to the working directory

In [2]:
workspace = os.path.join('steady-state')

if not os.path.exists(workspace):
    os.makedirs(workspace)
    
cwdpth = os.getcwd()

os.chdir(workspace)

### Create the MODFLOW model 
Store it (in this case in the variable `ml`, but you can call it whatever you want).  
The modelname will be the name given to all MODFLOW files (input and output).  
The exe_name should be the full path to your MODFLOW executable.  The version is either 'mf2k' for MODFLOW2000 or 'mf2005'for MODFLOW2005.

In [3]:
name = 'steady-state'

ml = flopy.modflow.Modflow(
    modelname=name, 
    exe_name='mf2005',
    version='mf2005'
)

### Discretization

We are defining all kind of discretization parameters.  

As we have a confining bed between top- and bottom-layer we have to specify  
the botm-parameter with three layer values `[50, 40, -10]` and set the   
laycbd-parameter with a flag (important: the flag must be 1) on the first layer `[1, 0]`




In [4]:
nlay = 2
nrow = ncol = 18
L = W = 18000
delr = L/nrow 
delc = W/ncol
top = 100
botm = [50, 40, -10]
laycbd = [1, 0]
itmuni = 1  # seconds

In [5]:
dis = flopy.modflow.ModflowDis(
    ml, 
    nlay=nlay, 
    nrow=nrow, 
    ncol=ncol, 
    delr=delr,
    delc=delc, 
    top=top, 
    botm=botm, 
    laycbd=laycbd,
    itmuni=itmuni
)

### BAS-Package 

Specify boundary conditions and starting heads. 

We can set one value for all cells.


In [6]:
ibound = 1
strt = 200

bas = flopy.modflow.ModflowBas(ml, ibound=ibound, strt=strt)

### LPF-Package
The aquifer properties (really only the hydraulic conductivity) are defined with the LPF package.

In [7]:
hk = [1E-4, 1E-5]
vkcb = 1E-7

lpf = flopy.modflow.ModflowLpf(ml, hk=hk, vkcb=vkcb)

### PCG-Package and OC-Package
Finally, we need to specify the solver we want to use (PCG with default values), and the output control (using the default values).  

Then we are ready to write all MODFLOW input files and run MODFLOW.

In [8]:
pcg = flopy.modflow.ModflowPcg(ml)
oc = flopy.modflow.ModflowOc(ml)
ml.write_input()
ml.run_model()

FloPy is using the following executable to run the model: /usr/local/bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: steady-state.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2018/05/28 10:57:59

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2018/05/28 10:57:59
 Elapsed run time:  0.005 Seconds

  Normal termination of simulation


(True, [])

### Read the heads-file

In [9]:
os.chdir(cwdpth)
hds = flopy.utils.HeadFile(os.path.join(workspace, name + '.hds'))

In [11]:
h = hds.get_data(kstpkper=(0, 0))
x = y = np.linspace(0, delr * nrow, nrow)

In [ ]:
%matplotlib inline
c = plt.contour(x, y, h[0], np.arange(90, 100.1, 0.2))
plt.clabel(c, fmt='%1.1f')
plt.axis('scaled')
plt.show()